English	English	en	6,519,984	56,055,628	1,090,881,532	1,034	43,816,165	119,491	893,446	1,123
French	français	fr	2,430,972	12,072,721	194,127,648	159	4,402,490	18,320	68,683	252
German	Deutsch	de	2,698,991	7,454,772	222,264,504	194	3,944,103	17,489	128,610	92
Japanese	日本語	ja	1,330,152	3,930,190	89,847,591	40	1,943,075	15,099	33,664	87
Spanish	español	es	1,783,096	7,675,975	143,773,321	63	6,569,908	14,720	0	204
Russian	русский	ru	1,828,528	7,376,221	123,027,378	76	3,201,398	11,037	236,620	153
Mandarin	中文	zh	1,283,503	7,062,397	71,878,020	66	3,231,960	9,091	59,906	206
Italian	italiano	it	1,759,047	7,449,115	127,630,202	120	2,263,048	8,544	142,345	179
Portuguese	português	pt	1,093,349	5,371,168	63,621,213	55	2,767,066	8,458	58,209	181
Persian	فارسی	fa	915,756	5,302,982	34,834,527	34	1,127,550	5,815	81,080	150


In [1]:
from bs4 import BeautifulSoup
import requests
import re, os, csv
import wikifunctions_s as wfs
import pandas as pd
import multiprocessing as mp

In [2]:
langs_10 = ['en','fr','de','ja','es','ru','zh','it','pt','fa']
langs_5 = ['en','fr','de','ja','es']

```
# get all pages in the Wikipedia namespace
rule_dfs_dict = {}
rule_dfs_list = []

for l in langs_10:
    #print(l)
    _temp = wfs.get_all_pages_in_namespace(l)
    rule_dfs_dict[l] = _temp
    rule_dfs_list.append(_temp)

len(rule_dfs_list)

# export each df in rule_dfs_list so that we don't have to get them all over again
output_directory = 'rules_list_all'

if not os.path.isdir(output_directory):
    os.mkdir(output_directory)

for df in rule_dfs_list:
    _lang = df.lang.values.tolist()[0]
    path = './{}/rule_list_{}.tsv'.format(output_directory,_lang)
    df.to_csv(path,index=False,sep='\t')
```

In [4]:
# now load those dfs and get all the ILLs for the pages
rule_dfs_dict = {}
rule_dfs_list = []

for l in langs_10:
    path = './rules_list_all/rule_list_{}.tsv'.format(l)
    _df = pd.read_csv(path, sep='\t',header=0)
    rule_dfs_dict[l] = _df
    rule_dfs_list.append(_df)

In [9]:
interlanguage_links = {}

for lang in langs_10:
    print(lang)
    interlanguage_links[lang] = {}
    # get the list of pages
    _df = rule_dfs_dict[lang]
    rule_list = _df['title'].values.tolist()

    for rule in rule_list: #[:10]:
        ill_dict = wfs.get_interlanguage_links(rule, endpoint=lang, redirects=1)
        interlanguage_links[lang][rule] = {}
        interlanguage_links[lang][rule]['count'] = len(list(ill_dict.keys()))
        interlanguage_links[lang][rule]['langs'] = list(ill_dict.keys())
        interlanguage_links[lang][rule]['links'] = ill_dict

en
fr
de
ja
es
ru
zh
it
pt
fa


In [16]:
import multiprocessing as mp

mp.cpu_count()

pool = mp.Pool(mp.cpu_count())

8